In [1]:
from bs4 import BeautifulSoup
import requests

import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 입력 셋팅

In [2]:
start_date = '20190901' # start와 end의 사이가 3의 배수만큼 존재 해야함
end_date = '20200223'

# 정확도 점수 삽입
p1_score = 0.5
p2_score = 0.3
p3_score = 0
p4_score = -0.3
p5_score = -0.5

# 가중치
d1_weight = 1
d2_weight = 0.8
d3_weight = 0.6
d4_weight = 0.4

## 3-1

In [3]:
URL = 'https://finance.naver.com/sise/entryJongmok.nhn?&page=' 
code_data=[]
for i in range(1,22,1):
        url = URL + str(i)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'lxml')
        items = soup.find_all('td',{'class':'ctg'})
        
        for item in items:
            txt = item.a.get('href')
            k = re.search('([\d]+)',txt)
            if k:
                code=k.group()
                name=item.text
                data=code,name
                code_data.append(data)
                
code_data = pd.DataFrame(code_data)
code_data.columns = ['code','name']
code_data.to_csv('data/ver3_1.csv')

## 3-2

In [4]:
code_data=pd.read_csv('data/ver3_1.csv')
code=[]
for i in range(len(code_data)):
    code.append(code_data[['code']].iloc[i][0])

original_data=pd.read_csv('data/data.csv')

df_close = pd.DataFrame()
df_date = []
for i in range(len(code)): 
    data = original_data
    data = original_data[original_data['code']==code[i]].reset_index().sort_values(by=['date'])
    data = data[['date','close']]
    data = data.set_index('date').loc['20170101':'20200223'] # 3의 배수 맞추기위해 날짜 조정
    df_close = pd.concat([df_close, data], axis=1)

df_close.columns = ['a{}' .format(i) for i in code]
# 현재 코스피 200 종목중 2년이상 존재 하지 않으면 삭제
df_close = df_close.dropna(axis=1) 

# 인덱스 변경해주기
# 원래 설정대로 날짜를 이용하면 그래프가 제대로 표현이 되지 않음.
df_date.append(df_close.index)
date=[]
for i in df_date[0]:
    date.append('{}-{}-{}'.format(str(i)[0:4],str(i)[4:6],str(i)[6:]))
df_close.index =date

dic={'aa':'c01','ab':'c02','ac':'c03','ad':'c04','ae':'c05',
     'ba':'c06','bb':'c07','bc':'c08','bd':'c09','be':'c10',
     'ca':'c11','cb':'c12','cc':'c13','cd':'c14','ce':'c15',
     'da':'c16','db':'c17','dc':'c18','dd':'c19','de':'c20',
     'ea':'c21','eb':'c22','ec':'c23','ed':'c24','ee':'c25'}



# <범위 정하기>
# return = ((종목[열][1] / 종목[열][0]) -1)*100
# 1. -0.3 < return < 0.3
# 2. 0.3 < return < 1.0
# 3. -1.0 < return < -0.3
# 4. return >1.0
# 5. return <-1.0

final_class=[]

for j in [i for i in df_close.columns]:
    stock = pd.DataFrame(np.array(df_close[j]).reshape(-1,3)).T
    stock.columns = ['b{}' .format(i) for i in range(len(stock.columns))]
    
    percentage=[]
    for k in stock.columns:
        for l in range(2):
            if ((stock[k][l+1]/stock[k][l])-1)*100 < -1.0:
                percentage.append('a')
            elif ((stock[k][l+1]/stock[k][l])-1)*100 >= -1.0  and ((stock[k][1]/stock[k][l])-1)*100 < -0.3:
                percentage.append('b')
            elif ((stock[k][l+1]/stock[k][l])-1)*100 >= -0.3 and ((stock[k][1]/stock[k][l])-1)*100 < 0.3:
                percentage.append('c')
            elif ((stock[k][l+1]/stock[k][l])-1)*100 >= 0.3 and ((stock[k][1]/stock[k][l])-1)*100 < 1.0:
                percentage.append('d')
            else :
                percentage.append('e')
    
    class_per = pd.DataFrame(np.array(percentage).reshape(-1,2)).T
    class_per.columns = ['b{}' .format(i) for i in range(len(stock.columns))]
    class_per.index = ['return_1','return_2']
    stock = pd.concat([stock, class_per], axis=0)
    
    class_=[]
    for m in range(stock.shape[1]):
        a=(stock.iloc[3,m]+stock.iloc[4,m])
        class_.append(dic[a])
    
    stock.loc[6]=class_
    stock.index = ['first_price','second_price','third_price','first_return','second_price','class']
    final_class.append(stock.loc['class',].tolist())
    
final = pd.DataFrame(np.array(final_class).reshape(len(df_close.columns),-1))
final.index = [i for i in df_close.columns]
final = final.T
final.to_csv('data/ver3_2.csv')

## 3.3

In [5]:
finla=pd.read_csv('data/ver3_2.csv')

code_list=[]
for i in [i for i in final.columns[1:]]:
    code_list.append(i)
    
data = final
code_list=[]
for i in [i for i in data.columns[1:]]:
    code_list.append(i)

In [6]:
actual_data=[]
pred_data=[]
num_code=[]

for codes in code_list:
    
    # 변수 설정
    code = codes
    n_test_set = 10 
    est_data = 2
    batch_size = 5
    
    train = np.array(data[code][:-n_test_set]) 
    test = data[-n_test_set:-est_data][code]
    class_set = sorted(set(train)) 
    class_set.append('NO')
    word_idx = {u:i for i, u in enumerate(class_set)}
    idx_word = np.array(class_set)
    text_as_int = np.array([word_idx[c] for c in train])
    seq_length = n_test_set 
    examples_per_epoch = len(text_as_int) // seq_length
    sentence_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) 
    sentence_dataset = sentence_dataset.batch(seq_length+est_data, drop_remainder=True) 
    def split(data): 
        return [data[:-est_data], data[-est_data]]
    train_dataset = sentence_dataset.map(split) 
    BATCH_SIZE = batch_size 
    steps_per_epoch = examples_per_epoch // BATCH_SIZE
    BUFFER_SIZE = 10000
    train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
    total_words = len(class_set)
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(total_words,n_test_set, input_length=seq_length),
        tf.keras.layers.LSTM(units=64, return_sequences=True),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.LSTM(units=32),
        tf.keras.layers.Dense(total_words, activation='softmax')])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    def testmodel(epoch, logs):
        if epoch % 5 != 0 and epoch != 49:
            return
        test_sentence = str(data[code][-20:].tolist()).replace("[","").replace("]","").replace(",","").replace("'","")

        next_words = 5
        for _ in range(next_words):
            test_text_X = test_sentence[-seq_length:]
            test_text_X = np.array([word_idx[c] if c in word_idx else word_idx['NO'] for c in test_text_X])
            test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding='pre', value=word_idx['NO'])

            output_idx = model.predict_classes(test_text_X)
            test_sentence += ' ' + idx_word[output_idx[0]]
    testmodelcb = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)
    history = model.fit(train_dataset.repeat(), epochs=100, steps_per_epoch=steps_per_epoch, callbacks=[testmodelcb], verbose=2)
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    test_sentence = str(data[code][-20:-2].tolist()).replace("[","").replace("]","").replace(",","").replace("'","")
    next_words = 2
    for _ in range(next_words):
        test_text_X = test_sentence.split(' ')[-seq_length:]
        test_text_X = np.array([word_idx[c] if c in word_idx else word_idx['NO'] for c in test_text_X])
        test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding='pre', value=word_idx['NO'])
        output_idx = model.predict_classes(test_text_X)
        test_sentence += ' ' + idx_word[output_idx[0]]
    num_code.append(codes)
    pred_data.append(test_sentence[-20:])
    actual_data.append(str(data[codes][-5:].tolist()).replace("[","").replace("]","").replace(",","").replace("'",""))

Train for 4 steps
Epoch 1/100
4/4 - 3s - loss: 2.7719 - accuracy: 0.0000e+00
Epoch 2/100
4/4 - 0s - loss: 2.7617 - accuracy: 0.2000
Epoch 3/100
4/4 - 0s - loss: 2.7503 - accuracy: 0.1500
Epoch 4/100
4/4 - 0s - loss: 2.7333 - accuracy: 0.1500
Epoch 5/100
4/4 - 0s - loss: 2.7094 - accuracy: 0.1500
Epoch 6/100
4/4 - 0s - loss: 2.6750 - accuracy: 0.1500
Epoch 7/100
4/4 - 0s - loss: 2.6187 - accuracy: 0.1500
Epoch 8/100
4/4 - 0s - loss: 2.5247 - accuracy: 0.1500
Epoch 9/100
4/4 - 0s - loss: 2.4526 - accuracy: 0.1500
Epoch 10/100
4/4 - 0s - loss: 2.4266 - accuracy: 0.1500
Epoch 11/100
4/4 - 0s - loss: 2.4087 - accuracy: 0.1500
Epoch 12/100
4/4 - 0s - loss: 2.3869 - accuracy: 0.1500
Epoch 13/100
4/4 - 0s - loss: 2.3633 - accuracy: 0.2000
Epoch 14/100
4/4 - 0s - loss: 2.3342 - accuracy: 0.2000
Epoch 15/100
4/4 - 0s - loss: 2.3385 - accuracy: 0.2000
Epoch 16/100
4/4 - 0s - loss: 2.3150 - accuracy: 0.2000
Epoch 17/100
4/4 - 0s - loss: 2.3150 - accuracy: 0.2000
Epoch 18/100
4/4 - 0s - loss: 2.296

Epoch 47/100
4/4 - 0s - loss: 0.7521 - accuracy: 0.8000
Epoch 48/100
4/4 - 0s - loss: 0.7952 - accuracy: 0.7000
Epoch 49/100
4/4 - 0s - loss: 0.7228 - accuracy: 0.8000
Epoch 50/100
4/4 - 0s - loss: 0.7051 - accuracy: 0.7500
Epoch 51/100
4/4 - 0s - loss: 0.5962 - accuracy: 0.8500
Epoch 52/100
4/4 - 0s - loss: 0.5762 - accuracy: 0.8500
Epoch 53/100
4/4 - 0s - loss: 0.6033 - accuracy: 0.8500
Epoch 54/100
4/4 - 0s - loss: 0.6421 - accuracy: 0.8000
Epoch 55/100
4/4 - 0s - loss: 0.5697 - accuracy: 0.8500
Epoch 56/100
4/4 - 0s - loss: 0.7184 - accuracy: 0.7500
Epoch 57/100
4/4 - 0s - loss: 0.6941 - accuracy: 0.7500
Epoch 58/100
4/4 - 0s - loss: 0.5848 - accuracy: 0.7500
Epoch 59/100
4/4 - 0s - loss: 0.6956 - accuracy: 0.7500
Epoch 60/100
4/4 - 0s - loss: 0.6222 - accuracy: 0.8500
Epoch 61/100
4/4 - 0s - loss: 0.4846 - accuracy: 0.9000
Epoch 62/100
4/4 - 0s - loss: 0.6427 - accuracy: 0.8000
Epoch 63/100
4/4 - 0s - loss: 0.5209 - accuracy: 0.8500
Epoch 64/100
4/4 - 0s - loss: 0.4531 - accuracy:

Epoch 94/100
4/4 - 0s - loss: 0.9750 - accuracy: 0.7000
Epoch 95/100
4/4 - 0s - loss: 0.7293 - accuracy: 0.7500
Epoch 96/100
4/4 - 0s - loss: 0.9848 - accuracy: 0.7000
Epoch 97/100
4/4 - 0s - loss: 0.7812 - accuracy: 0.7500
Epoch 98/100
4/4 - 0s - loss: 0.7066 - accuracy: 0.8000
Epoch 99/100
4/4 - 0s - loss: 0.7708 - accuracy: 0.7500
Epoch 100/100
4/4 - 0s - loss: 0.7506 - accuracy: 0.8000
Train for 4 steps
Epoch 1/100
4/4 - 3s - loss: 2.7705 - accuracy: 0.1000
Epoch 2/100
4/4 - 0s - loss: 2.7575 - accuracy: 0.0500
Epoch 3/100
4/4 - 0s - loss: 2.7451 - accuracy: 0.1000
Epoch 4/100
4/4 - 0s - loss: 2.7259 - accuracy: 0.1000
Epoch 5/100
4/4 - 0s - loss: 2.6950 - accuracy: 0.1000
Epoch 6/100
4/4 - 0s - loss: 2.6568 - accuracy: 0.1000
Epoch 7/100
4/4 - 0s - loss: 2.5756 - accuracy: 0.1000
Epoch 8/100
4/4 - 0s - loss: 2.4856 - accuracy: 0.1000
Epoch 9/100
4/4 - 0s - loss: 2.4240 - accuracy: 0.1000
Epoch 10/100
4/4 - 0s - loss: 2.3805 - accuracy: 0.1000
Epoch 11/100
4/4 - 0s - loss: 2.3468 -

Epoch 40/100
4/4 - 0s - loss: 1.5844 - accuracy: 0.5500
Epoch 41/100
4/4 - 0s - loss: 1.5345 - accuracy: 0.5000
Epoch 42/100
4/4 - 0s - loss: 1.5262 - accuracy: 0.4500
Epoch 43/100
4/4 - 0s - loss: 1.5025 - accuracy: 0.4500
Epoch 44/100
4/4 - 0s - loss: 1.3689 - accuracy: 0.6000
Epoch 45/100
4/4 - 0s - loss: 1.3379 - accuracy: 0.6000
Epoch 46/100
4/4 - 0s - loss: 1.2780 - accuracy: 0.6500
Epoch 47/100
4/4 - 0s - loss: 1.2330 - accuracy: 0.6500
Epoch 48/100
4/4 - 0s - loss: 1.2075 - accuracy: 0.6000
Epoch 49/100
4/4 - 0s - loss: 1.1599 - accuracy: 0.6000
Epoch 50/100
4/4 - 0s - loss: 1.1204 - accuracy: 0.6500
Epoch 51/100
4/4 - 0s - loss: 1.0636 - accuracy: 0.6500
Epoch 52/100
4/4 - 0s - loss: 1.0887 - accuracy: 0.6500
Epoch 53/100
4/4 - 0s - loss: 1.0618 - accuracy: 0.6500
Epoch 54/100
4/4 - 0s - loss: 1.0122 - accuracy: 0.6500
Epoch 55/100
4/4 - 0s - loss: 1.0290 - accuracy: 0.6500
Epoch 56/100
4/4 - 0s - loss: 1.0555 - accuracy: 0.6500
Epoch 57/100
4/4 - 0s - loss: 1.0153 - accuracy:

4/4 - 0s - loss: 0.3986 - accuracy: 0.9500
Epoch 87/100
4/4 - 0s - loss: 0.3882 - accuracy: 1.0000
Epoch 88/100
4/4 - 0s - loss: 0.3803 - accuracy: 1.0000
Epoch 89/100
4/4 - 0s - loss: 0.3759 - accuracy: 1.0000
Epoch 90/100
4/4 - 0s - loss: 0.3750 - accuracy: 0.9500
Epoch 91/100
4/4 - 0s - loss: 0.3606 - accuracy: 1.0000
Epoch 92/100
4/4 - 0s - loss: 0.3572 - accuracy: 1.0000
Epoch 93/100
4/4 - 0s - loss: 0.3508 - accuracy: 1.0000
Epoch 94/100
4/4 - 0s - loss: 0.3449 - accuracy: 1.0000
Epoch 95/100
4/4 - 0s - loss: 0.3413 - accuracy: 1.0000
Epoch 96/100
4/4 - 0s - loss: 0.3512 - accuracy: 1.0000
Epoch 97/100
4/4 - 0s - loss: 0.3609 - accuracy: 0.9500
Epoch 98/100
4/4 - 0s - loss: 0.3851 - accuracy: 0.9500
Epoch 99/100
4/4 - 0s - loss: 0.3277 - accuracy: 1.0000
Epoch 100/100
4/4 - 0s - loss: 0.3335 - accuracy: 1.0000
Train for 4 steps
Epoch 1/100
4/4 - 3s - loss: 2.7734 - accuracy: 0.0000e+00
Epoch 2/100
4/4 - 0s - loss: 2.7661 - accuracy: 0.1000
Epoch 3/100
4/4 - 0s - loss: 2.7600 - ac

...

Epoch 19/100
4/4 - 0s - loss: 2.1687 - accuracy: 0.2000
Epoch 20/100
4/4 - 0s - loss: 2.1679 - accuracy: 0.2500
Epoch 21/100
4/4 - 0s - loss: 2.1651 - accuracy: 0.2000
Epoch 22/100
4/4 - 0s - loss: 2.1549 - accuracy: 0.2000
Epoch 23/100
4/4 - 0s - loss: 2.1574 - accuracy: 0.2000
Epoch 24/100
4/4 - 0s - loss: 2.1417 - accuracy: 0.2000
Epoch 25/100
4/4 - 0s - loss: 2.1376 - accuracy: 0.2000
Epoch 26/100
4/4 - 0s - loss: 2.1409 - accuracy: 0.1500
Epoch 27/100
4/4 - 0s - loss: 2.1296 - accuracy: 0.2500
Epoch 28/100
4/4 - 0s - loss: 2.1178 - accuracy: 0.2500
Epoch 29/100
4/4 - 0s - loss: 2.1111 - accuracy: 0.2000
Epoch 30/100
4/4 - 0s - loss: 2.0959 - accuracy: 0.3000
Epoch 31/100
4/4 - 0s - loss: 2.0871 - accuracy: 0.3500
Epoch 32/100
4/4 - 0s - loss: 2.0594 - accuracy: 0.3500
Epoch 33/100
4/4 - 0s - loss: 2.0397 - accuracy: 0.3500
Epoch 34/100
4/4 - 0s - loss: 2.0102 - accuracy: 0.3000
Epoch 35/100
4/4 - 0s - loss: 1.9805 - accuracy: 0.3500
Epoch 36/100
4/4 - 0s - loss: 1.9533 - accuracy:

Epoch 66/100
4/4 - 0s - loss: 0.9823 - accuracy: 0.6500
Epoch 67/100
4/4 - 0s - loss: 0.9249 - accuracy: 0.7000
Epoch 68/100
4/4 - 0s - loss: 0.9382 - accuracy: 0.7500
Epoch 69/100
4/4 - 0s - loss: 1.0015 - accuracy: 0.7500
Epoch 70/100
4/4 - 0s - loss: 0.9779 - accuracy: 0.7000
Epoch 71/100
4/4 - 0s - loss: 1.0331 - accuracy: 0.7000
Epoch 72/100
4/4 - 0s - loss: 0.9520 - accuracy: 0.7500
Epoch 73/100
4/4 - 0s - loss: 1.0953 - accuracy: 0.6000
Epoch 74/100
4/4 - 0s - loss: 0.9584 - accuracy: 0.7000
Epoch 75/100
4/4 - 0s - loss: 0.8966 - accuracy: 0.7000
Epoch 76/100
4/4 - 0s - loss: 0.9532 - accuracy: 0.7000
Epoch 77/100
4/4 - 0s - loss: 0.9041 - accuracy: 0.6500
Epoch 78/100
4/4 - 0s - loss: 0.8785 - accuracy: 0.8000
Epoch 79/100
4/4 - 0s - loss: 0.8426 - accuracy: 0.7000
Epoch 80/100
4/4 - 0s - loss: 0.8251 - accuracy: 0.7500
Epoch 81/100
4/4 - 0s - loss: 0.7952 - accuracy: 0.7500
Epoch 82/100
4/4 - 0s - loss: 0.7772 - accuracy: 0.7500
Epoch 83/100
4/4 - 0s - loss: 0.7778 - accuracy:

Epoch 12/100
4/4 - 0s - loss: 2.1412 - accuracy: 0.1500
Epoch 13/100
4/4 - 0s - loss: 2.1242 - accuracy: 0.2000
Epoch 14/100
4/4 - 0s - loss: 2.1330 - accuracy: 0.2000
Epoch 15/100
4/4 - 0s - loss: 2.1224 - accuracy: 0.2000
Epoch 16/100
4/4 - 0s - loss: 2.1051 - accuracy: 0.2000
Epoch 17/100
4/4 - 0s - loss: 2.0953 - accuracy: 0.2000
Epoch 18/100
4/4 - 0s - loss: 2.1014 - accuracy: 0.2000
Epoch 19/100
4/4 - 0s - loss: 2.0866 - accuracy: 0.2000
Epoch 20/100
4/4 - 0s - loss: 2.0887 - accuracy: 0.2000
Epoch 21/100
4/4 - 0s - loss: 2.0902 - accuracy: 0.2000
Epoch 22/100
4/4 - 0s - loss: 2.0738 - accuracy: 0.2000
Epoch 23/100
4/4 - 0s - loss: 2.0774 - accuracy: 0.2000
Epoch 24/100
4/4 - 0s - loss: 2.0703 - accuracy: 0.2000
Epoch 25/100
4/4 - 0s - loss: 2.0615 - accuracy: 0.2000
Epoch 26/100
4/4 - 0s - loss: 2.0642 - accuracy: 0.2000
Epoch 27/100
4/4 - 0s - loss: 2.0625 - accuracy: 0.2000
Epoch 28/100
4/4 - 0s - loss: 2.0404 - accuracy: 0.2000
Epoch 29/100
4/4 - 0s - loss: 2.0466 - accuracy:

Epoch 59/100
4/4 - 0s - loss: 0.6976 - accuracy: 0.8500
Epoch 60/100
4/4 - 0s - loss: 0.6702 - accuracy: 0.8500
Epoch 61/100
4/4 - 0s - loss: 0.6591 - accuracy: 0.8500
Epoch 62/100
4/4 - 0s - loss: 0.6401 - accuracy: 0.9000
Epoch 63/100
4/4 - 0s - loss: 0.5969 - accuracy: 0.9000
Epoch 64/100
4/4 - 0s - loss: 0.5803 - accuracy: 0.9000
Epoch 65/100
4/4 - 0s - loss: 0.6633 - accuracy: 0.8000
Epoch 66/100
4/4 - 0s - loss: 0.9401 - accuracy: 0.8000
Epoch 67/100
4/4 - 0s - loss: 0.6257 - accuracy: 0.8000
Epoch 68/100
4/4 - 0s - loss: 1.0340 - accuracy: 0.7500
Epoch 69/100
4/4 - 0s - loss: 0.8531 - accuracy: 0.8000
Epoch 70/100
4/4 - 0s - loss: 0.6268 - accuracy: 0.9000
Epoch 71/100
4/4 - 0s - loss: 0.8568 - accuracy: 0.7000
Epoch 72/100
4/4 - 0s - loss: 0.9920 - accuracy: 0.7000
Epoch 73/100
4/4 - 0s - loss: 0.7469 - accuracy: 0.7500
Epoch 74/100
4/4 - 0s - loss: 0.5174 - accuracy: 0.9500
Epoch 75/100
4/4 - 0s - loss: 0.6018 - accuracy: 0.9000
Epoch 76/100
4/4 - 0s - loss: 0.5833 - accuracy:

Epoch 5/100
4/4 - 0s - loss: 2.7204 - accuracy: 0.2000
Epoch 6/100
4/4 - 0s - loss: 2.6973 - accuracy: 0.2000
Epoch 7/100
4/4 - 0s - loss: 2.6569 - accuracy: 0.2000
Epoch 8/100
4/4 - 0s - loss: 2.5932 - accuracy: 0.2000
Epoch 9/100
4/4 - 0s - loss: 2.5106 - accuracy: 0.2000
Epoch 10/100
4/4 - 0s - loss: 2.4061 - accuracy: 0.2000
Epoch 11/100
4/4 - 0s - loss: 2.3536 - accuracy: 0.2000
Epoch 12/100
4/4 - 0s - loss: 2.3098 - accuracy: 0.2000
Epoch 13/100
4/4 - 0s - loss: 2.2684 - accuracy: 0.2000
Epoch 14/100
4/4 - 0s - loss: 2.2480 - accuracy: 0.2000
Epoch 15/100
4/4 - 0s - loss: 2.2068 - accuracy: 0.2000
Epoch 16/100
4/4 - 0s - loss: 2.1981 - accuracy: 0.1500
Epoch 17/100
4/4 - 0s - loss: 2.1931 - accuracy: 0.2000
Epoch 18/100
4/4 - 0s - loss: 2.1700 - accuracy: 0.2000
Epoch 19/100
4/4 - 0s - loss: 2.1618 - accuracy: 0.2000
Epoch 20/100
4/4 - 0s - loss: 2.1514 - accuracy: 0.2000
Epoch 21/100
4/4 - 0s - loss: 2.1532 - accuracy: 0.2000
Epoch 22/100
4/4 - 0s - loss: 2.1418 - accuracy: 0.20

In [7]:
a=pd.DataFrame(actual_data)
a.columns=['actual_data']
b=pd.DataFrame(pred_data)
b.columns=['pred_data']
c=pd.DataFrame(num_code)
c.columns=['code']
df_class=pd.concat([c,a,b],axis=1)
df_class=df_class.set_index('code')
df_class.to_csv('data/ver3_3.csv')

# 3.4

In [8]:
data=pd.read_csv('data/ver3_3.csv')

#새로운 dic 생성하기
dic_2={'c01':'aa','c02':'ab','c03':'ac','c04':'ad','c05':'ae',
       'c06':'ba','c07':'bb','c08':'bc','c09':'bd','c10':'be',
       'c11':'ca','c12':'cb','c13':'cc','c14':'cd','c15':'ce',
       'c16':'da','c17|':'db','c18':'dc','c19':'dd','c20':'de',
       'c21':'ea','c22':'eb','c23':'ec','ed':'c24','c25':'ee'}

dic_3={'a':'-1.0', 'b':'-1.0,-0.3','c':'-0.3,0.3','d':'0.3,1.0','e':'1.0'}

# 예측 클래스를 분류해주기
actual_class=[]
pred_class=[]
for i in range(len(data)):
    actual = data['actual_data'][i].split()[-2:]
    actual_class.append(actual)
    pred = data['pred_data'][i].split()[-2:]
    pred_class.append(pred)
data['actual_data'] = actual_class
data['pred_data'] = pred_class

actual_class=[]
pred_class=[]
for i in range(len(data)):
    actual_class.append(dic_2[data['actual_data'][i][0]])
    actual_class.append(dic_2[data['actual_data'][i][1]])
for j in range(len(data)):
    pred_class.append(dic_2[data['pred_data'][j][0]])
    pred_class.append(dic_2[data['pred_data'][j][1]])
    
data['actual_class']=np.array(actual_class).reshape(-1,2).tolist()
data['pred_class']=np.array(pred_class).reshape(-1,2).tolist()

#각각의 예측과 실제값을 묶어서 4개로 나누기
a=[]
b=[]
c=[]
d=[]
for i in range(len(data)):
    aa=[data['actual_class'][i][0][0],data['pred_class'][i][0][0]]
    a.append(aa)
    bb=[data['actual_class'][i][0][1],data['pred_class'][i][0][1]]
    b.append(bb)  
    cc=[data['actual_class'][i][1][0],data['pred_class'][i][1][0]]
    c.append(cc)   
    dd=[data['actual_class'][i][1][1],data['pred_class'][i][1][1]]
    d.append(dd) 
data['first_class']=a
data['second_class']=b
data['third_class']=c
data['forth_class']=d

# 새로운 data 생성
data=data[['code','first_class','second_class','third_class','forth_class']]

# 새로운 class 만들어주기
p1=[['a','a'],['b','b'],['c','c'],['d','d'],['e','e']] 
p2=[['a','b'],['b','a'],['b','c'],['c','b'],['c','d'],['d','c'],['d','e'],['e','d']]
p3=[['a','c'],['c','a'],['b','d'],['d','b'],['c','e'],['e','c']]
p4=[['a','d'],['d','a'],['b','e'],['e','b']] 
p5=[['a','e'],['e','a']]

# 점수 매기는 함수 만들어주기
def sumin(data):
    score=[]
    for i in data.index :
        acc=0
        #first class
        if data['first_class'][i] in p1:
            acc += p1_score*d1_weight

        elif data['first_class'][i] in p2:
            acc += p2_score*d1_weight

        elif data['first_class'][i] in p3:
            acc += p3_score*d1_weight

        elif data['first_class'][i] in p4:
            acc += p4_score*d1_weight

        elif data['first_class'][i] in p5:
            acc += p5_score*d1_weight
            
        #second class
        if data['second_class'][i] in p1:
            acc += p1_score*d2_weight

        elif data['second_class'][i] in p2:
            acc += p2_score*d2_weight

        elif data['second_class'][i] in p3:
            acc += p3_score*d2_weight

        elif data['second_class'][i] in p4:
            acc += p4_score*d2_weight

        elif data['second_class'][i] in p5:
            acc += p5_score*d2_weight

        # third class
        if data['third_class'][i] in p1:
            acc += p1_score*d3_weight

        elif data['third_class'][i] in p2:
            acc += p2_score*d3_weight

        elif data['third_class'][i] in p3:
            acc += p3_score*d3_weight

        elif data['third_class'][i] in p4:
            acc += p4_score*d3_weight

        elif data['third_class'][i] in p5:
            acc += p5_score*d3_weight
            
        # forth class
        if data['forth_class'][i] in p1:
            acc += p1_score*d3_weight

        elif data['forth_class'][i] in p2:
            acc += p2_score*d3_weight

        elif data['forth_class'][i] in p3:
            acc += p3_score*d3_weight

        elif data['forth_class'][i] in p4:
            acc += p4_score*d3_weight

        elif data['forth_class'][i] in p5:
            acc += p5_score*d3_weight      
        score.append(round(acc,3))
    return score

final_score=pd.concat([pd.DataFrame(data['code']),pd.DataFrame(sumin(data))],axis=1)
final_score.columns = ['code', 'score']
final_score.sort_values(by='score', ascending=False)[:10]['code']
code_list = final_score.sort_values(by='score', ascending=False)[:10]['code'].values.tolist()

# 최종 결과

In [9]:
final_score

,code,score
0,a660,0.02
1,a35420,0.60
2,a51910,-0.80
3,a5380,0.50
4,a68270,-0.20
5,a6400,0.00
6,a28260,0.48
7,a51900,0.00
8,a12330,1.18
9,a17670,0.48
